# A Python Program to Solve a Sumaddle puzzle

## Author: Owen Chen
## Last Modified: 4/3/2023

## Run Time:
O( n!*(n-1)!*(n-2)!...2!*1!)

## Sumaddle Puzze
A Sumaddle puzzle is to fill numbers into a nxn grid with constraints.  For each row and each column of a size n, you need to fill them with these numbers exactly once:

- 0, 0, 1, 2, 3, ..., n-1

The sum of the numbers between two 0s in each row must equal to a row sum given on the right side of the table.  

Similiarly, the sum of the numbers between two 0s in each column must equal to a column sum given on the bottom of the table.  

## Inventor
The puzzle was first invented by Kazunori Saito, who contributed an example to the Japan Puzzle Championship 2006. He named it ビトゥイーンサム, meaning "between sum".


## Example
- Here is a 5x5 example:

<center><img src="https://sumaddle.com/uploads/1/3/6/1/136147655/published/examplepuzzle.png?1666011815"></center>

The blocks greyed out are the blockers with a value of 0.

## Basic rules:

1. Every row and every column must contain exactly two filled squares ("blocks").
2. Remaining squares contain the numbers 1, 2, 3 (etc.), with each number appearing exactly once in every row and in every column.
3. A value is shown at the end of a row or at the base of a column, the numbers in the squares between the two blocks of that row or column must add up to the value shown.
4. A sum value of 0 is a special case for rule 3. If a row or column has a sum of 0, it means the two blocks must be next to each other, with no intervening squares.


See the detailed description of the puzzle in 
- https://sumaddle.com/fundamentals.html


In [ ]:
## Version 3- changes from V2:

- add a global dictionary map for a split from a list into two sub lists.  This will be used to keep track what numbers being used and what numbers remaining
- add a global dictionary map for every permutation with its block sum value

In [15]:
import time
import numpy as np
from tqdm import tqdm
from itertools import combinations, permutations

#Global Variables for two dictionary maps

SUBSET_SPLIT_MAP = {}

ROW_SUM_MAP = {}

GRID = np.full([7,7], -1, dtype =int)

def get_subset_list(n, parent_row):
    fulllist = [0] + [i for i in range(n-1)]
    return  [e for e in fulllist if not e in parent_row or parent_row.remove(e)]

def initialize_subset_split_map(n):
    global SUBSET_SPLIT_MAP
    SUBSET_SPLIT_MAP = {}
    fullset = set(i for i in range(n-1))
    fulllist = [0] + [i for i in range(n-1)]
    for i in range(1, n):
        combos = [val for val in combinations(fulllist, i)]
        for subset in combos:
            subset = tuple(sorted(subset))
            if subset not in SUBSET_SPLIT_MAP:
                SUBSET_SPLIT_MAP[subset] = subtract_a_set(fullset, subset)

def subtract_a_set(fullset:set, subset:tuple) -> set:
    """
    Subtract a fullset all emlements from a given subset
    Return a subset of remaining elements
    """
    zerocount = subset.count(0)    
    remaining_set = fullset - set(subset)
    if zerocount < 2:
        return remaining_set.union({0})
    else:
        return remaining_set
        
    
def initialize_row_sum_map(n):
    global ROW_SUM_MAP
    ROW_SUM_MAP = {}
    fullset_tuple = (0,) + tuple(i for i in range(n-1))
    perms = [val for val in permutations(fullset_tuple, n)] 
    for perm in perms:
        zeroindex = perm.index(0) + 1
        if perm[zeroindex] == 0:
            ROW_SUM_MAP[perm] = 0
        else:
            row_sum = 0
            for k in perm[zeroindex:]:
                if k == 0:
                     ROW_SUM_MAP[perm] = row_sum
                     break
                else:
                     row_sum += k

def initialize_grid(n):
    global GRID
    GRID =  np.full([n,n], -1, dtype=int)

    
# Check patterns of each row

def check_zero_zero(row):   
    return ROW_SUM_MAP[row] == 0

def check_blocksum(row, s):
    return ROW_SUM_MAP[row] == s


def get_firstrow_permutations(row_sum):
	"""
	Get the first row permutations with row_sum as a constraint only
	"""
	if row_sum >= 0:
		return [ perm for perm, sum in ROW_SUM_MAP.items() if sum == row_sum]
	else:
		return list(ROW_SUM_MAP.keys())
		
def get_firstcol_permutations(col_sum, first):
	"""
	Get the first column permutations with col_sum as a constraint and the first element is first.
	"""
	if col_sum >= 0:
		return [ perm for perm, sum in ROW_SUM_MAP.items() if sum == col_sum and perm[0] == first]
	else:
		return [ perm for perm in ROW_SUM_MAP.values() if perm[0] == first]
 
def get_num_list(n, row_list, col_list):  
    zeros_allowed = 1
    if row_list:
        zeros_in_row = row_list.count(0)
        if col_list:
            if type(col_list) is list:
                zero_in_col = col_list.count(0)
                constraint = set(row_list + tuple(col_list))
            else:
                if col_list == 0:
                    zero_in_col = 1
                else:
                    zero_in_col = 0
                constraint = set(row_list + (col_list,))
            if  zero_in_col > 1 or zeros_in_row > 1:                
                num_list = tuple(k for k in range(1, n-1) if k not in constraint)          
            else:
                num_list = (0,) + tuple(k for k in range(1, n-1) if k not in constraint)     
        else:
            if  zeros_in_row > 1:
                num_list = tuple(k for k in range(1, n-1) if k not in row_list) 
            else:
                num_list = (0,) + tuple(k for k in range(1, n-1) if k not in row_list) 
    elif col_list:
        if type(col_list) is list:
            zero_in_col = col_list.count(0)
        else:
            if col_list == 0:
                zero_in_col = 1
            else:
                zero_in_col = 0
            col_list = list(col_list)
        if zero_in_col > 1:
            num_list = tuple(k for k in range(1, n-1) if k not in col_list)       
        else:
            num_list = (0,) + tuple(k for k in range(1, n-1) if k not in col_list)
                 
    else:
            num_list = tuple(i for i in range(n-1))        
        
    return num_list


def get_last_num(n, exclude_list):    
    num_list =  [i for i in range(n-1) if i not in exclude_list]
    if len(num_list):
        return num_list[0]
    else:
        return 0    
    
def check_row_constraint(row, row_sum): 
    if row in ROW_SUM_MAP:
        if row_sum >=0:
            return ROW_SUM_MAP[row] == row_sum
        else:
            return True
    else:
        return False
    
def get_firstrow_permutations(k, row_sum, first, row_numbers, row_block):    
    """
    Get the first row permutations:
    - row_sum as a constraint for a row sum
    - first as the first element
    - row_numbers as the list of available numbers
    - row_block the number of zero blocks in the parent
    """
    if row_sum == 0:
        if row_block > 1:
            # something wrong
            return None
        elif row_block == 1:
            if first > 0:
                #something wrong
                return None
            else:
                # no constraint
                if len(row_numbers) < k:
                    row_numbers.append([0])
                perms = [ val for val in permutations(row_numbers, k)]
                return list(set(perms))
        else:
            # find consecutive zeros only
            if row_numbers.count(0)  == 0:
                # something wrong
                return None
            else:
                #Create permutations with a single 0
                temp_perms = [ val for val in permutations(row_numbers, k)]
                #Insert a second zero next to first zero
                perms = []
                for perm in temp_perms:
                    zeroindex = perm.index(0)
                    perm.insert(zeroindex, 0)
                    perms.append(perm)
                return perms
            
    elif row_sum > 0:
        if row_block > 1:
            # something wrong
            return None
        elif row_block == 1:
            if row_numbers.count(0) != 1:
                # something wrong
                return None
            if first > row_sum:
                #something wrong
                return None
            else:
                row_sum -= first
                temp_perms = [ val for val in permutations(row_numbers, k)]
                if row_sum == 0:
                    #first number must be 0
                    return [val for val in temp_perms if val[0] == 0]
                else:
                    #Get a positive accumulative sum to row_sum
                    perms = []
                    for perm in temp_perms:
                        zeroindex = perm.index(0)
                        if row_sum == sum(perm[:zeroindex]):
                            perms.append(perm)
                    return perms
        else:
            # find consecutive zeros only
            if row_numbers.count(0)  == 0:
                # something wrong
                return None
            else:
                #Create permutations with a single 0
                temp_perms = [ val for val in permutations(row_numbers, k)]
                #Insert a second zero next to first zero
                perms = []
                for perm in temp_perms:
                    zeroindex = perm.index(0)
                    perm.insert(zeroindex, 0)
                    perms.append(perm)
                return perms        
    else:
        # no constraint
        if len(row_numbers) < k:
            row_numbers.append([0])
        perms = [ val for val in permutations(row_numbers, k)]
        return list(set(perms))        

    
def get_firstcol_permutations(k, first, col_sum, col_numbers, col_block):    
    """
    Get the first col permutations:
    - col_sum as a constraint for a col sum
    - first as the first element
    - col_numbers as the list of available numbers
    - col_block the number of zero blocks in the parent
    """
    
    if first > 0:
        col_numbers = [val for val in col_numbers if val != first]
    else:
        if col_block > 0:
            col_numbers = [val for val in col_numbers if val > 0]
        col_block += 1    
        
    if col_sum == 0:
        if col_block > 1:
            # something wrong
            return None
        elif col_block == 1:
            if first > 0:
                #something wrong
                return None
            else:
                # no constraint
                if len(col_numbers) < k:
                    col_numbers.append([0])
                perms = [ (first,) + val for val in permutations(col_numbers, k)]
                return list(set(perms))
        else:
            # find consecutive zeros only
            if col_numbers.count(0)  == 0:
                # something wrong
                return None
            else:
                #Create permutations with a single 0
                temp_perms = [ val for val in permutations(col_numbers, k)]
                #Insert a second zero next to first zero
                perms = []
                for perm in temp_perms:
                    zeroindex = perm.index(0)
                    perm.insert(zeroindex, 0)
                    perms.append(perm)
                return perms
            
    elif col_sum > 0:
        if col_block > 1:
            # something wrong
            return None
        elif col_block == 1:
            if col_numbers.count(0) != 1:
                # something wrong
                return None
            if first > col_sum:
                #something wrong
                return None
            else:
                col_sum -= first
                temp_perms = [ val for val in permutations(col_numbers, k-1)]
                if col_sum == 0:
                    #first number must be 0
                    return [(first,) + val for val in temp_perms if val[0] == 0]
                else:
                    #Get a positive accumulative sum to col_sum
                    perms = []
                    for perm in temp_perms:
                        zeroindex = perm.index(0)
                        if col_sum == sum(perm[:zeroindex]):
                            perms.append((first,) + perm)
                    return perms
        else:
            if col_numbers.count(0)  == 0:
                # something wrong
                return None
            else:
                if first == 0:
                    #Create permutations with one 0s                    
                    temp_perms = [ val for val in permutations(col_numbers, k-1)]
                    perms = []
                    for perm in temp_perms:
                        zeroindex = perm.index(0)
                        if col_sum == sum(perm[:zeroindex]):
                            perms.append((first,) + perm)
                    return perms
                else:
                    #Create permutations with two 0s
                    col_numbers = [ val for val in col_numbers if val != first]
                    col_numbers.append(0)
                    temp_perms = [ val for val in permutations(col_numbers, k-1)]
                    perms = []
                    for perm in temp_perms:
                        zeroindex = perm.index(0)
                        lsum = 0
                        for i in range(zeroindex+1, k):
                            if perm[i] > 0:
                                lsum += perm[i]
                            else:
                                if col_sum == lsum:
                                    perms.append((first,) + perm)
                                else:
                                    break
                    return perms  
    else:
        # no col sum constraint
        if first > 0:
            col_numbers = [val for val in col_numbers if val != first]
        else:
            if col_blocks > 0:
                col_numbers = [val for val in col_numbers if val > 0 ]            
            
        if len(col_numbers) < k-1:
            col_numbers.append([0])
        perms = [ (first,) + val for val in permutations(col_numbers, k-1)]
        return list(set(perms))   

def check_sol_row_constraint(k, sol, row_sums, row_zeroblocks):
    satisfied = False
    for i in range(k):
        # Row constraint
        satisfied = False
        row = sol[i]
        # only need to check row_sum == 0 or >0  (< 0 means no constraint)
        if row_sums[i] == 0:
            if row_zeroblocks[i] == 0:
                for j in range(1,k):
                    if row[j-1] == 0 and row[j] == 0:
                        satisfied = True
                        break
            elif row_zeroblocks[i] == 1:
                if row[0] == 0:
                    satisfied = True
            else:
                satisfied = True
        elif row_sums[i] > 0:
            if row_zeroblocks[i] == 0:
                temp_sum = 0
                zeroindex = row.index(0)
                for j in range(zeroindex+1,k):
                    if row[j] > 0:
                        temp_sum += row[j]
                    else:
                        break
                if temp_sum == row_sums[i]:
                    satisfied = True
            elif row_zeroblocks[i] == 1:
                temp_sum = 0
                for j in range(k):
                    if row[j] > 0:
                        temp_sum += row[j]
                    else:
                        break
                if temp_sum == row_sums[i]:
                    satisfied = True    
            else:
                return False
        else:
            satisfied = True
        
    return satisfied
        
        
def check_sol_constraints(k, sol, row_sums, col_sums, row_zeroblocks, col_zeroblocks):
    satisfied = check_sol_row_constraint(k,sol, row_sums, row_zeroblocks)
    if satisfied:
        return check_sol_row_constraint(k,sol.T, col_sums, col_zeroblocks)
    else:
        return False

    
def merge_sub_sol(k, r0, c0, sub_sol):
    sol = np.full([k,k], -1)
    sol[0] = r0
    sol[:, 0] = c0
    sol[1:,1:] = sub_sol
    return sol
                                                      
def print_matrix(sol):
    for col in sol:
        print(col)

def solve_sub_puzzle(k, row_sums, col_sums, row_numbers, col_numbers, row_blocks, col_blocks):
    subgrid = np.full((k,k), -1)
    if k == 2:
        row_perms = [[row_numbers[0][0], row_numbers[0][1]], [row_numbers[0][1], row_numbers[0][0]] ]
        col_perms = [[col_numbers[0][0], col_numbers[0][1]], [col_numbers[0][1], col_numbers[0][0]] ]
        for row0 in row_perms[0]:             
            for col0 in col_perms[0]:
                if col0[0] == row0[0]:
                    last_row = [ row for row in row_perms if row[0] == col0[1] ]                    
                    if last_row:
                        subgrid[0] = row0
                        subgrid[1] = last_row
                        if check_sol_constraints(k, subgrid,row_sums, col_sums, row_blocks, col_blocks):                 
                            return subgrid                        
        return None
    
    else:    
        sub_row_sums = row_sums[1:]
        sub_col_sums = col_sums[1:]
        sub_row_blocks = row_blocks[1:]
        sub_col_blocks = col_blocks[1:]
        sub_row_numbers = row_numbers[1:]
        sub_col_numbers = col_numbers[1:]
        rows0 = get_firstrow_permutations(k, row_sums[0], row_numbers[0], row_blocks[0])
        for r0 in rows0:                                
            subgrid[0] = r0
            cols0 =  get_firstcol_permutations(k,r0[0], col_sums[0], col_numbers[0], col_blocks[0])
            for c0 in cols0:
                subgrid[:,0] = c0     
                for i in range(1, k):                    
                    # update row_sums, row_numbers, col_numbers        
                    if c0[i] > 0:   
                        if row_sums[i] > 0:
                            if row_blocks[i] == 1:
                                if  c0[i] > row_sums[i]:
                                    #constraint violation
                                    break
                                else:
                                    sub_row_sums[i-1] -= c0[i+1]                        
                        elif row_sums[i] == 0:
                            if row_blocks[i+1] > 0:
                                #constraint violation
                                break
                        sub_row_number[i-1] = [num for num in row_number[i] if num != c0[i]]
                        
                    else:
                        if row_blocks[i] == 2:
                            #constraint violation
                            break 
                        elif row_blocks[i] == 1:
                            if  row_sums[i] > 0:
                                #constraint violation
                                break     
                            else:
                                sub_row_blocks[i-1] +=1                            
                                sub_row_sums[i-1] = -1
                                sub_row_number[i-1] = [num for num in row_number[i] if num > 0]
                        else:
                                sub_row_blocks[i-1] +=1                            

                    # update col_sums, col_numbers col_blocks
                    if r0[i] > 0:                   
                        if col_sums[i] > 0:
                            if col_blocks[i] == 1:
                                if  c0[i] > col_sums[i]:
                                    #constraint violation
                                    break
                                else:
                                    sub_col_sums[i-1] -= c0[i+1]                        
                        elif col_sums[i] == 0:
                            if col_blocks[i+1] > 0:
                                #constraint violation
                                break
                        sub_col_number[i-1] = [num for num in col_number[i] if num != r0[i]]
                        
                    else:
                        if col_blocks[i] == 2:
                            #constraint violation
                            break 
                        elif col_blocks[i] == 1:
                            if  col_sums[i] > 0:
                                #constraint violation
                                break     
                            else:
                                sub_col_blocks[i-1] +=1                            
                                sub_col_sums[i-1] = -1
                                sub_col_number[i-1] = [num for num in col_number[i] if num > 0]
                        else:
                                sub_col_blocks[i-1] +=1     
                                                
                sub_sol = solve_sub_puzzle(k-1,sub_row_sums,sub_col_sums,sub_row_numbers, sub_col_numbers,sub_row_blocks, sub_col_blocks)
                if sub_sol:
                    subgrid[1:, 1:] = sub_sol
                    if check_sol_constraints(k, subgrid, row_sums, col_sums, row_blocks, col_blocks):
                        return subgrid                
        return None

def solve_sumaddle_puzzle(row_sums, col_sums):
    n = len(row_sums)
    initialize_subset_split_map(n)
    initialize_row_sum_map(n)   
    initialize_grid(n)
    row_numbers = [[i for i in range(n-1)] for j in range(n)]
    col_numbers = row_numbers
    row_zeroblocks = [ [0] for j in range(n)]
    col_zeroblocks = row_zeroblocks
    sol = solve_sub_puzzle(n, row_sums, col_sums, row_numbers, col_numbers, row_zeroblocks, col_zeroblocks)    
    #sol = generate_solutions(row_sums,col_sums, n)    
    if sol is not None:
        print(f"Solution for a {n}x{n} with row constraint:{row_sums}, and column constraint:{col_sums}")
        print(sol)
    else:
        print("No solution")    

In [16]:
import time
start_time = time.time()
solve_sumaddle_puzzle([3, 1, 0, 2], [3, 0, 0,0])
print(f"Run time: {time.time() - start_time} seconds")

NameError: name 'first_col' is not defined

In [290]:
import time
start_time = time.time()
solve_sumaddle_puzzle([5, 0, 5, 6, 0], [0, 6, 3, 0, 3])
print(f"Run time: {time.time() - start_time} seconds")

 75%|█████████████████████████████████████████████████████████                   | 3/4 [00:00<00:00, 939.94it/s]

Solution for a 5x5 with row constraint:[5, 0, 5, 6, 0], and column constraint:[0, 6, 3, 0, 3]
[[1 0 3 2 0]
 [3 1 0 0 2]
 [0 3 2 0 1]
 [0 2 1 3 0]
 [2 0 0 1 3]]
Run time: 0.005195140838623047 seconds


In [291]:
start_time = time.time()
solve_sumaddle_puzzle([5, 1, 0, 5, 0], [2, 1, 0, 0, 6])
print(f"Run time: {time.time() - start_time} seconds")

 75%|████████████████████████████████████████████████████████▎                  | 3/4 [00:00<00:00, 3166.31it/s]

Solution for a 5x5 with row constraint:[5, 1, 0, 5, 0], and column constraint:[2, 1, 0, 0, 6]
[[1 0 3 2 0]
 [0 1 0 3 2]
 [2 0 0 1 3]
 [0 3 2 0 1]
 [3 2 1 0 0]]
Run time: 0.0043773651123046875 seconds


In [292]:
start_time = time.time()
solve_sumaddle_puzzle([9, 2, 4, -1 ,-1 , 3],  [7, 10, 2, -1,-1 , -1])
print(f"Run time: {time.time() - start_time} seconds")

 92%|███████████████████████████████████████████████████████████████████▊      | 11/12 [00:00<00:00, 575.75it/s]

Solution for a 6x6 with row constraint:[9, 2, 4, -1, -1, 3], and column constraint:[7, 10, 2, -1, -1, -1]
[[1 0 4 3 2 0]
 [0 2 0 4 3 1]
 [3 1 2 0 4 0]
 [4 3 0 1 0 2]
 [0 4 1 2 0 3]
 [2 0 3 0 1 4]]
Run time: 0.023044824600219727 seconds


In [293]:
start_time = time.time()
solve_sumaddle_puzzle([10, 0, 9, 1 ,3, 7, 3], [0, 11, 0, 8, 14, 9, 15])
print(f"Run time: {time.time() - start_time} seconds")

 93%|████████████████████████████████████████████████████████████████████▏    | 112/120 [18:56<01:21, 10.15s/it]

Solution for a 7x7 with row constraint:[10, 0, 9, 1, 3, 7, 3], and column constraint:[0, 11, 0, 8, 14, 9, 15]
[[5 0 3 4 1 2 0]
 [4 5 2 0 0 1 3]
 [2 3 0 5 4 0 1]
 [0 1 0 3 5 4 2]
 [0 2 1 0 3 5 4]
 [3 0 4 1 2 0 5]
 [1 4 5 2 0 3 0]]
Run time: 1136.7590901851654 seconds
